In [1]:
#二階booking.com
#台北市-四人(只要換網址就可以變成2人)
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pymongo
import time
import random
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException, StaleElementReferenceException

client=pymongo.MongoClient("mongodb+srv://帳號:密碼@mongodb連線設定")

db=client.資料庫名稱

collection=db.資料表名稱


def choose_checkin():
    
    driver.find_elements_by_css_selector('button[class="d47738b911 fb1847d86a"]')[0].click()
    time.sleep(1)
    checkin_date=driver.find_elements_by_css_selector('td[class="b21c1c6c83 e505d9d049"]')[1].get_attribute("innerText")
    checkin_month=driver.find_elements_by_css_selector('div[class="ab0d1629e5"]')[0].get_attribute("innerText")[:2]
    time.sleep(1)
    #選日期
    driver.find_elements_by_css_selector('td[class="b21c1c6c83 e505d9d049"]')[1].click()
    #確認
    driver.find_element_by_css_selector('button[class="fc63351294 a822bdf511 d4b6b7a9e7 f7db01295e af18dbd5a4 f4605622ad c827b27356"]').click()

    return checkin_month,checkin_date


desired_capabilities = DesiredCapabilities.CHROME  #調整selinium讀取方式
desired_capabilities["pageLoadStrategy"] = "none" #直接使用DOM樹，進行操作
driver = webdriver.Chrome("C:\\Users\\Lihome\\chromedriver.exe")
booking="https://www.booking.com/searchresults.zh-tw.html?ss=%E5%8F%B0%E5%8C%97&ssne=%E5%8F%B0%E5%8C%97&ssne_untouched=%E5%8F%B0%E5%8C%97&label=gog235jc-1DCAso5wFCE2VzbGl0ZS1ob3RlbC10YWlwZWlIMFgDaOcBiAEBmAEwuAEXyAEM2AED6AEB-AECiAIBqAIDuALrjqKUBsACAdICJGExOTQyYTg0LTg5MDItNGM0My04NWJhLTBmNjMxMDM0OGMyMdgCBOACAQ&sid=d84e42d213d58ca1489b5345202df287&aid=356980&highlighted_hotels=814737&lang=zh-tw&sb=1&src_elem=sb&src=searchresults&dest_id=-2637882&dest_type=city&checkin=2022-06-04&checkout=2022-06-05&group_adults=4&no_rooms=1&group_children=0&sb_travel_purpose=leisure"
driver.get(booking)


checkin_month="6"
checkin_date="4"
tiapei_dist=["松山","信義","大安","中山","中正","大同","萬華","文山","南港","內湖","士林","北投"]


while True:

    if checkin_month=="6 " and checkin_date=="11":
        break
    else:

        #找一頁的筆數
        data_count=len(driver.find_elements_by_css_selector('div[class="d20f4628d0"]'))
        time.sleep(3)
        #找最後一頁的數字
        last_page=driver.find_elements_by_css_selector('li[class="f32a99c8d1"]')[-1].get_attribute("innerText")

        #開始抓資料
        for x in range(int(last_page)): 
            for hotel in range(data_count):
                
                
                # 地址，確認是不是台北的行政區
                try:
                    address = driver.find_elements_by_css_selector('span[data-testid="address"]')[hotel].get_attribute("innerText")
                    if address[0:2] not in tiapei_dist:
                        continue
                    else:
                        address=address.replace(" ","")
                        if address[2]==",": # 判斷鄉鎮市區 沒區字
                            address = address[:2]+"區"
                        elif address[2]=="區":
                            address = address[:3]
                        elif address[:3]=="西門町": # 判斷鄉鎮市區 沒區字
                            address=address.replace("西門町","萬華區")
                except :
                    continue
                    
                #其他縣市需要這樣設定
#                  if address[1]=="區": # 判斷鄉鎮市區 沒區字
#                     address = address[:2]
#                 elif address=="台中":
#                     continue
#                 elif address[2]=="區":
#                     address = address[:3]
                
                        
                # 飯店名稱
                try:
                    if  driver.find_elements_by_css_selector('div[class="fcab3ed991 a23c043802"]')[hotel].get_attribute("innerText")=="是美精品飯店":
                        continue
                    else:
                        name = driver.find_elements_by_css_selector('div[class="fcab3ed991 a23c043802"]')[hotel].get_attribute("innerText")
                        print(name)
                except :
                    continue

                # 網址
                hotel_link = driver.find_elements_by_css_selector('a[class="e13098a59f"]')[hotel].get_attribute("href")


                #入住日期
                if len(checkin_month)<2:
                    if len(checkin_date)<2:
                        day="2022-0%s-0%s"%(checkin_month,checkin_date)
                    else:
                        day="2022-0%s-%s"%(checkin_month,checkin_date)
                else:
                    if len(checkin_date)<2:
                        day="2022-%s-0%s"%(checkin_month,checkin_date)
                    else:
                        day="2022-%s-%s"%(checkin_month,checkin_date)

                # 房型
                bed_num=driver.find_elements_by_css_selector('div[class="d22a7c133b"]')[hotel].get_attribute("innerText")
                if "2 張單人床" in bed_num:
                    roomtype=driver.find_elements_by_css_selector('span[class="df597226dd"]')[hotel].get_attribute("innerText")
                    roomtype=roomtype+"(2張床)"

                else:
                    roomtype=driver.find_elements_by_css_selector('span[class="df597226dd"]')[hotel].get_attribute("innerText")


                # 價格
                price = driver.find_elements_by_css_selector('span[class="fcab3ed991 bd73d13072"]')[hotel]
                price = price.text.replace("TWD ","")
                price = price.replace(",","")
                price = int(price)
                

                #二階
                driver.find_elements_by_css_selector('a[class="fc63351294 a822bdf511 d4b6b7a9e7 f7db01295e f4605622ad b2f0d6a80e"]')[hotel].is_displayed()
                enter =driver.find_elements_by_css_selector('a[class="fc63351294 a822bdf511 d4b6b7a9e7 f7db01295e f4605622ad b2f0d6a80e"]')[hotel]
                enter.click()
                num = driver.window_handles
                driver.switch_to.window(num[-1])
                time.sleep(2)
                driver.execute_script("window.stop();")#網頁停止loading
                
                time.sleep(3)
                #地圖連結
                try:
                    map_link=driver.find_element_by_css_selector('a[id="hotel_address"]').get_attribute("href")
                except:
                    driver.close()
                    driver.switch_to.window(num[0])
                    continue

                #停車資訊
                try:
                    car_park=driver.find_elements_by_css_selector('div[class="bui-spacer--medium hotel-facilities-group__policy"]')[1].get_attribute("innerText")
                    if car_park=="住宿方于客房提供WiFi（免費）。":
                        car_park=driver.find_elements_by_css_selector('div[class="bui-spacer--medium hotel-facilities-group__policy"]')[2].get_attribute("innerText")
                        if car_park[:5]=="無法預約：":
                            car_park_state="Yes"
                            car_park=car_park[6:-1]
                                
                        elif car_park[:5]=="不需預約：":
                            car_park_state="Yes"
                            car_park=car_park[6:-1]
                                
                        elif car_park[:4]=="需預約：":
                            car_park_state="Yes"
                            car_park=car_park[5:-1]


                        elif car_park=="無停車設施。":
                            car_park=car_park[:-1]
                            car_park_state="No"
                            
                        elif car_park=="提供停車相關設施":
                            car_park_state="Yes"

                    elif car_park[:5]=="無法預約：":
                        car_park_state="Yes"
                        car_park=car_park[6:-1]
                        
                    elif car_park[:5]=="不需預約：":
                        car_park_state="Yes"
                        car_park=car_park[6:-1]
                            
                    elif car_park[:4]=="需預約：":
                        car_park_state="Yes"
                        car_park=car_park[5:-1]
                            
                    elif car_park=="提供停車相關設施":
                        car_park_state="Yes"
                        
                    elif car_park=="無停車設施。":
                        car_park=car_park[:-1]
                        car_park_state="No"

                    
                except:
                    driver.close()
                    driver.switch_to.window(num[0])
                    continue


                 # 評分
                try:
                    location=driver.find_elements_by_css_selector('span[class="c-score-bar__score"]')[1].get_attribute("innerText")
                    location=float(location)
                    cleanliness=driver.find_elements_by_css_selector('span[class="c-score-bar__score"]')[2].get_attribute("innerText")
                    cleanliness=float(cleanliness)
                    comfort=driver.find_elements_by_css_selector('span[class="c-score-bar__score"]')[4].get_attribute("innerText")
                    comfort=float(comfort)
                    
                except:
                    driver.close()
                    driver.switch_to.window(num[0])

                    continue
                
                data={
                    "Name":name,
                    "District":address,
                    "Link":hotel_link,
                    "Checkin_day":day,
                    "Roomtype":roomtype,
                    "Price":price,
                    "Map_link":map_link,
                    "Car_Park_State":car_park_state,
                    "Car_Park":car_park,
                    "Location":location,
                    "Cleanliness":cleanliness,
                    "Comfort":comfort
                    }
        
                result=collection.insert_one(data)
                
                time.sleep(2)
                driver.close()
                driver.switch_to.window(num[0])


            # 下一頁
            try:
                last_page_button = driver.find_element_by_css_selector('li[class="f32a99c8d1 ebd02eda9e"]').get_attribute("innerText")
                if last_page_button==last_page:
                    continue
                else:
                    next_page = driver.find_element_by_css_selector('button[aria-label="下一頁"]').click()
                    time.sleep(random.randint(5,10))
                    print(last_page_button)  #印出已完成頁數

            except :
                print("不知道")
                
        checkin_month,checkin_date=choose_checkin()

print("結束")


C:\Users\Lihome\AppData\Local\Temp/ipykernel_27424/1688080622.py:38: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\\Users\\Lihome\\chromedriver.exe")
C:\Users\Lihome\AppData\Local\Temp/ipykernel_27424/1688080622.py:55: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  data_count=len(driver.find_elements_by_css_selector('div[class="d20f4628d0"]'))
C:\Users\Lihome\AppData\Local\Temp/ipykernel_27424/1688080622.py:59: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  last_page=driver.find_elements_by_css_selector('li[class="f32a99c8d1"]')[-1].get_attribute("innerText")
C:\Users\Lihome\AppData\Local\Temp/ipykernel_27424/1688080622.py:69: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELE

誠品行旅


C:\Users\Lihome\AppData\Local\Temp/ipykernel_27424/1688080622.py:139: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  map_link=driver.find_element_by_css_selector('a[id="hotel_address"]').get_attribute("href")
C:\Users\Lihome\AppData\Local\Temp/ipykernel_27424/1688080622.py:147: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  car_park=driver.find_elements_by_css_selector('div[class="bui-spacer--medium hotel-facilities-group__policy"]')[1].get_attribute("innerText")
C:\Users\Lihome\AppData\Local\Temp/ipykernel_27424/1688080622.py:198: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  location=driver.find_elements_by_css_selector('span[class="c-score-bar__score"]')[1].get_attribute("innerText")
C:\Users\Lihome\AppData\Lo

德立莊酒店
台北凱達大飯店
富驛時尚酒店- 南京東路館
台北天成大飯店
Hotel M 台北摩莎精品旅店
Hotel PaPa Whale
台北長榮桂冠酒店
怡品商旅
台北花園大酒店
洛碁大飯店忠孝館
洛碁大飯店南京館
凱旋酒店
天成文旅華山町- 華山文創園區
凱統飯店
阿樹國際旅店
路境行旅
丹迪旅店- 天母店
城市商旅- 南東館
洛碁大飯店山水閣
蒲園飯店
JR東日本大飯店 台北
嘉賓閣溫泉會館


C:\Users\Lihome\AppData\Local\Temp/ipykernel_27424/1688080622.py:235: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  last_page_button = driver.find_element_by_css_selector('li[class="f32a99c8d1 ebd02eda9e"]').get_attribute("innerText")
C:\Users\Lihome\AppData\Local\Temp/ipykernel_27424/1688080622.py:239: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  next_page = driver.find_element_by_css_selector('button[aria-label="下一頁"]').click()


...

天津大飯店 
天雲旅棧 台北中山
Hotelpoispois泡泡飯店
雅莊商務旅館
泊居旅店
士林讚
新驛旅店台北車站一館
凡登東棧商務旅店
北投麗禧溫泉酒店
13
誠品行旅
V-one - 寧夏2號旅店
山樂溫泉飯店
吉泰旅店 
探索汽車旅館-延平舘
台北寒舍艾美酒店
國聯大飯店
樂玩旅
客來思樂行旅
宜家居商旅
和怡商旅 (原法皇)
台北唯客樂飯店
沃客商旅西門館
仲信大湖商旅
品格子旅店北車館
貝斯特旅店
玩客棧Player Hostel
文舺行旅
第一大飯店
金都精緻溫泉飯店
14
誠品行旅
新舍商旅林森館
友徠精品旅館
正旅館 (信義)
T.O. Hotel 台北車站
天使青旅台北西門
情覓精品汽車旅館
水都溫泉會館
茄子青旅
門草行旅
北門窩泊旅
君品酒店
千彩格精品旅店
建山大旅社
甲山林湯旅
旅居文旅-西門驛站
在家行旅At Home Hotel
凡登台北商務旅店
迪樂商旅(非防疫旅館)
台北旅‧晨行旅
復新文旅
泊客青旅(碩瀚行旅)
15
誠品行旅
紫園旅店
探索汽車旅館-南港館
OrigInn Space
東驛商旅 西寧館
詩漫精品旅馆
葫蘆旅社
伊倫商務旅館
鴨川旅館
幸福蘋果旅店
豪悅大飯店
思泊客台北101
星盒青年旅館
東驛商務旅館
凱微精品旅館
台北和璞飯店
龍山旅社
東鑫商務旅館
長富時尚旅店
東姿商務旅館
16
誠品行旅
斯格加旅店
力歐時尚旅館西門館
璽愛旅店
力歐時尚旅館站前館
茶街民宿
梅樓驛站
台北發現青年旅舍
橙舍青旅(台北一館)
途中‧台北國際青年旅舍
門埕青旅 
Dan Hostel丹居青旅
鉑泊客旅店
小角落 Corner Hostel & Cafe
橙舍太空艙旅館


C:\Users\Lihome\AppData\Local\Temp/ipykernel_27424/1688080622.py:23: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_elements_by_css_selector('button[class="d47738b911 fb1847d86a"]')[0].click()
C:\Users\Lihome\AppData\Local\Temp/ipykernel_27424/1688080622.py:25: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  checkin_date=driver.find_elements_by_css_selector('td[class="b21c1c6c83 e505d9d049"]')[1].get_attribute("innerText")
C:\Users\Lihome\AppData\Local\Temp/ipykernel_27424/1688080622.py:26: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  checkin_month=driver.find_elements_by_css_selector('div[class="ab0d1629e5"]')[0].get_attribute("innerText")[:2]
C:\Users\Lihome\AppData\Local\Temp/ipykernel_27424/1688

誠品行旅
千彩格精品旅店
德立莊酒店
台北凱達大飯店
富驛時尚酒店- 南京東路館
台北天成大飯店
怡品商旅
天成文旅華山町- 華山文創園區
台北花園大酒店
台北長榮桂冠酒店
Hotel PaPa Whale
洛碁大飯店忠孝館
丹迪旅店- 天母店
凱統飯店
洛碁大飯店南京館
凱旋酒店
路境行旅
洛碁大飯店山水閣
新驛旅店 - 復興北路店
阿樹國際旅店
老爺會館台北南西-中山站
老爺會館台北林森 – 華山文創園區
1
誠品行旅
城市商旅- 南東館
苓旅
新驛旅店西門捷運店
嘉賓閣溫泉會館
蒲園飯店
谷墨商旅
台北六福萬怡酒店
台北萬豪酒店
北投輕行旅
你好咖啡旅館
陽明溫泉度假村
捷絲旅西門町館
喜瑞飯店
台北王朝大酒店
儷客旅店-台北車站館 
美亞商旅
優美飯店-雙連MRT
泉都溫泉會館
悠趣旅店
西門日記-六福館
神旺大飯店
方舟旅店-長安復興館
洛碁大飯店松山館
2
誠品行旅
華亭棧
新驛旅店台北車站三館
艾爾行旅 
印石時尚旅館
JR東日本大飯店 台北
台北福華大飯店
雅璞文旅
旅樂序精品旅館站前一館
沃旅
西門好好玩
葳皇時尚飯店
旅城驛鎮
日光拾玖 
所在行旅
美福大飯店
北投亞太飯店 
新驛旅店台北車站二館
信星青年旅館- 台北車站
方舟旅店 - 東門館
合星青年旅館 
謙匯-普樂室行旅
慕居行旅
儷夏商旅 - 西門捷運館
3
誠品行旅
五倉行館
城市商旅站前館
南陽街壹號旅店
家賓旅店
鳳凰閣溫泉飯店
陽明山美國渡假村
趣旅館．林森館
囍閱文旅
沐House
泊居旅店二館
五月家台大館
台北天禾旅店
台北亞都麗緻大飯店
mrhost北行旅
初見輕旅
米尼旅店
大新商旅
西悠飯店台北店
愛行旅
台北歐華酒店
意芙旅店
Ecfa Hotel - Wan Nian
4
誠品行旅
悠逸商旅台北士林
台湾青旅膠囊旅店
Sleep Inn B&B
西門日記旅店
瑪奇文旅
北投日勝生加賀屋國際溫泉飯店
AS Suite輕奢旅宿-西門捷運站3分鐘3 mins to Ximen MRT
安和精品商務旅館
晶華酒店
晴悅旅店
喜園旅店
台北遠東香格里拉
丰居旅店 - 忠孝館
RF富裕自由旅店 - 林森館
錦棧•旅
沃田旅店
仙渡莊旅社
約克設計旅店
長春商務旅館 
北投麗禧溫泉酒店
睡覺盒子青年旅館
5
誠品行旅
儷夏商旅- 西門町西寧館
能量旅店
森浩文旅-STEP IN

尚印2館
PlainStay 城西時光
木文陶喜時尚旅館
北門文旅
悅心旅
T.O. Hotel 承德館
函舍旅館
采舍精品旅館
大稻埕花園旅店
東皇渡假旅店
尚印旅店
古山園旅社
拉開門輕旅 Le Passage Hostel
東門3號膠囊旅店
金弘旅店
西門町有間書房--廉宿也有大選擇
皇家玫瑰旅館林森館
璞邸城市膠囊行旅
8
誠品行旅
國王大飯店
慕舍酒店
西門大飯店
西門町作家居--家大不加價
文林居 
台北艾特文旅西門町
Goodscompany Inn
曲水會館
欣欣旅店
Xinyi AnheMRT3min,CBD Garden view
旅莊商務飯店
峻美商旅
東龍大飯店
內江商旅
北投春天酒店
萬九126公寓
一等好旅店
千慧商務旅館
美觀園旅社
忠孝敦化民宿
台北綠蒂飯店
陽光溫暖萬三館
小王子甜蜜旅店
Ms. One
9
誠品行旅
八德185公寓
皇家玫瑰旅館新生館
富陽
趣西門旅店
兒童樂園民宿
西門大飯店B
轉角寓到你
八二 Family Villa
Fuxing Second
豪贊旅店
Ximending Tokyo & Contemporary House
品格子旅店- 西門館
新三12公寓
永八
Taipei's time square - Ximending 
Piano House & BR4
" 錢三"
松菸553公寓
幸福旅店
八一
延三
和苑三井花園飯店台北忠孝
兄弟大飯店
10
誠品行旅
台北西門町意舍
丹迪旅店大安森林公園店
洛碁大飯店舞衣南京館
台北時代寓所
台北大倉久和大飯店
台北君悅酒店
台北國泰萬怡酒店
北投天玥泉會館
洛碁大飯店南港館
宜家商旅
台北士林萬麗酒店
台北W 酒店
台北東旅
寒居酒店
華山文旅
南港老爺行旅
日野苑
藝宿商旅 - 台北 (寧夏館)
台北喜來登大飯店
格拉斯麗台北飯店
天閣酒店 台北劍潭
金普頓大安台北-洲際酒店集團酒店
町記憶旅店3館
11
誠品行旅
陽明山中國麗緻大飯店
二十輪旅店大安館
寒舍艾麗酒店
和逸飯店．台北民生館
北城大飯店
松哖酒店
天閣酒店台北士林
樂居微風
台北圓山大飯店
北投老爺酒店
RF富裕自由商旅 - 忠孝館
台北美侖大飯店
華泰瑞舍
神旺商務酒店
台北宏都金殿飯店
燦路都飯店
二十輪旅店西門町館
密都飯店
天閣酒店台北復興
薆悅酒店
台北老爺大酒店
清翼居采風

台北時代寓所
10
誠品行旅
台北國泰萬怡酒店
洛碁大飯店南港館
丹迪旅店大安森林公園店
北投天玥泉會館
宜家商旅
台北W 酒店
台北大倉久和大飯店
台北士林萬麗酒店
寒居酒店
華山文旅
南港老爺行旅
藝宿商旅 - 台北 (寧夏館)
台北喜來登大飯店
天閣酒店 台北劍潭
日野苑
金普頓大安台北-洲際酒店集團酒店
台北東旅
陽明山中國麗緻大飯店
格拉斯麗台北飯店
町記憶旅店3館
和逸飯店．台北民生館
北投老爺酒店
寒舍艾麗酒店
天閣酒店台北士林
11
誠品行旅
華泰瑞舍
北城大飯店
松哖酒店
台北美侖大飯店
樂居微風
台北圓山大飯店
家賓旅店
二十輪旅店大安館
密都飯店
神旺商務酒店
燦路都飯店
RF富裕自由商旅 - 忠孝館
天閣酒店台北復興
清翼居采風館
天閣酒店台北南西
柯達大飯店台北敦南
台北文華東方酒店
台北宏都金殿飯店
立德北投倆人旅店
華泰王子大飯店
慕軒飯店
二十輪旅店西門町館
台北旅坊
台北怡亨酒店
12
誠品行旅
薆悅酒店
大地酒店
台北老爺大酒店
城市商旅 北門館
友統大飯店
卡夫人背包客棧 台北車站青年館
漫映旅舍
天雲旅棧 台北中山
蜂巢旅店-台北車站館
南港安樂旅社
福容大飯店台北一館
維多麗亞酒店
貴族商務飯店
薇閣精品旅館 - 大直旗艦館
正旅館
臺北中山九昱希爾頓逸林酒店
仙渡莊旅社
富園國際商務飯店
沃客汽車旅館
儷夏商旅- 西門町西寧館
麗都唯客樂飯店
士林讚
13
誠品行旅
泊居旅店
新驛旅店台北車站一館
Hotelpoispois泡泡飯店
台北寒舍艾美酒店
梅樓驛站
雅莊商務旅館
凡登東棧商務旅店
樂玩旅
V-one - 寧夏2號旅店
吉泰旅店 
沃客商旅西門館
探索汽車旅館-延平舘
正旅館 (信義)
台北唯客樂飯店
T.O. Hotel 台北車站
金都精緻溫泉飯店
香城大飯店松山店
國聯大飯店
客來思樂行旅
宜家居商旅
14
誠品行旅
仲信大湖商旅
文舺行旅
第一大飯店
門草行旅
貝斯特旅店
喬合大飯店
新舍商旅林森館
天使青旅台北西門
品格子旅店北車館
友徠精品旅館
水都溫泉會館
台北旅‧晨行旅
情覓精品汽車旅館
茄子青旅
北門窩泊旅
旅居文旅-西門驛站
建山大旅社
天津大飯店 
凡登台北商務旅店
君品酒店
甲山林湯旅
復新文旅
15
誠品行旅
泊客青旅(碩瀚行旅)
迪樂商旅(非防疫旅館)
東驛商旅 西寧館
紫園旅店
Ori